# Импорты

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
import warnings
warnings.filterwarnings("ignore")

In [29]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import KBinsDiscretizer

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
import optuna

In [31]:
from scipy import stats
from association_metrics import CramersV
import phik
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
import pickle

In [32]:
import random
import os

In [33]:
import torch
import tensorflow as tf

# Константы

In [37]:
def set_all_seeds(seed):
    
    # python's seeds
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

    # torch's seeds
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # tensorflow's seed
    tf.random.set_seed(seed)
    
SEED = 2021
set_all_seeds(seed=SEED)

# 2.1 🎓🐍 Пишем и организуем код

## 📚 Структура директории 📂

In [ ]:
# Работая локально или на сервере, создай себе хотя бы базовые вспомогательные директории:

# |---competition/
#     |---data/ — исходные данные
#     |---notebooks/ — ноутбуки 
#     |---models/ — обученные файлы
#     |---subs/ — сабмишн файлы
#     |---tmp_data/ — временные файлы, которые по окончанию соревнования можно смело удалять
#     |---src/ — вспомогательные .py файлы

In [ ]:
# ├── LICENSE            <- Open-source license if one is chosen
# ├── Makefile           <- Makefile with convenience commands like `make data` or `make train`
# ├── README.md          <- The top-level README for developers using this project.
# ├── data
# │   ├── external       <- Data from third party sources.
# │   ├── interim        <- Intermediate data that has been transformed.
# │   ├── processed      <- The final, canonical data sets for modeling.
# │   └── raw            <- The original, immutable data dump.
# │
# ├── docs               <- A default mkdocs project; see www.mkdocs.org for details
# │
# ├── models             <- Trained and serialized models, model predictions, or model summaries
# │
# ├── notebooks          <- Jupyter notebooks. Naming convention is a number (for ordering),
# │                         the creator's initials, and a short `-` delimited description, e.g.
# │                         `1.0-jqp-initial-data-exploration`.
# │
# ├── pyproject.toml     <- Project configuration file with package metadata for 
# │                         {{ cookiecutter.module_name }} and configuration for tools like black
# │
# ├── references         <- Data dictionaries, manuals, and all other explanatory materials.
# │
# ├── reports            <- Generated analysis as HTML, PDF, LaTeX, etc.
# │   └── figures        <- Generated graphics and figures to be used in reporting
# │
# ├── requirements.txt   <- The requirements file for reproducing the analysis environment, e.g.
# │                         generated with `pip freeze > requirements.txt`
# │
# ├── setup.cfg          <- Configuration file for flake8
# │
# └── {{ cookiecutter.module_name }}   <- Source code for use in this project.
#     │
#     ├── __init__.py             <- Makes {{ cookiecutter.module_name }} a Python module
#     │
#     ├── config.py               <- Store useful variables and configuration
#     │
#     ├── dataset.py              <- Scripts to download or generate data
#     │
#     ├── features.py             <- Code to create features for modeling
#     │
#     ├── modeling                
#     │   ├── __init__.py 
#     │   ├── predict.py          <- Code to run model inference with trained models          
#     │   └── train.py            <- Code to train models
#     │
#     └── plots.py                <- Code to create visualizations 

## Сохраняй порядок колонок в моделях

In [ ]:
# Оказывается, если в модель (например LightGBM) при фиксированных сидах 
# подавать признаки в разной последовательности, то результаты будут очень сильно разные!

# features = np.unique(features)

## Сохраняй результаты вычисления, если они долго считались

In [ ]:
# regenerate = True  # Флаг, который ты легко можешь переключать

# if regenerate: # При первом проходе готовим данные
#      df = make_features(df) # Подготавливаем признаки для трейна
#      df.to_csv('train_with_features.csv', index=False) # Сохраняем
#      sub = make_features(sub) # Подготавливаем признаки для сабмита
#      sub.to_csv('test_with_features.csv', index=False) # Сохраняем

# else: # При повторном проходе можно сэкономить бесценное время
#      df = pd.read_csv('train_with_features.csv')
#      sub = pd.read_csv('test_with_features.csv')

## Сохраняй файлы пиклом (df.to_pickle()/pd.read_pickle())

In [ ]:
# import pickle

# a = {'hello': 'world'}

# with open('filename.pickle', 'wb') as handle:
#     pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)

# print(a == b)

## Используй классы вместо словарей

In [ ]:
# class Paths:
#     model_xgb = '../models/xgb.pickle'
#     model_cb  = '../models/cb.pickle'
#     model_lgb = '../models/lgb.pickle'
    
# paths = Paths()

# paths.model_xgb  # вместо paths['model_xgb'] при работе со словарем

## Режим работы

In [ ]:
# Сделай два режима работы своего ноутбука/скрипта: validation и test. 
# Первым режимом будешь пользоваться для локального обучения и тестирования, 
# вторым для обучения на всех данных, предсказания на тестовой выборке и получения файла с предсказанием.

In [ ]:
# Например:

# X = pd.read_csv('data/train.csv')
# y = pd.read_csv('data/y_train.csv')

# MODE = 'validation'
# if MODE == 'validation':
#     X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                         test_size=0.2,
#                                                         random_state=42)
# elif MODE == 'test':
#     X_train = X
#     y_train = y
#     X_test = pd.read_csv('data/test.csv')

## Используй tqdm для «подсветки» выполнения кода

In [44]:
import time

from tqdm import tqdm

for i in tqdm(range(10)):
    time.sleep(0.05)

100%|██████████| 10/10 [00:00<00:00, 19.76it/s]


# Tail